In [1]:
# Import required libraries
import boto3
import pandas as pd
import time
from datetime import datetime

In [2]:
# AWS Configuration
AWS_REGION = 'us-east-1'
DATABASE = 'a05'
TABLE = 'a05'

In [5]:
# Create Athena client
athena_client = boto3.client('athena', region_name=AWS_REGION)

# Function to execute Athena query
def execute_athena_query(query, output_location=None):
    """
    Execute an Athena query and return results as DataFrame.
    
    Args:
        query: SQL query string
        output_location: S3 location for query results
        
    Returns:
        pandas DataFrame with query results
    """
    # Generate unique output location if not provided
    if output_location is None:
        timestamp = str(int(time.time()))
        output_location = f"s3://athena-cg1372/query-results/{timestamp}/"
    
    # Start query execution
    response = athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': DATABASE},
        ResultConfiguration={'OutputLocation': output_location}
    )
    
    query_id = response['QueryExecutionId']
    print(f"Started query execution: {query_id}")
    
    # Wait for query completion
    while True:
        response = athena_client.get_query_execution(QueryExecutionId=query_id)
        status = response['QueryExecution']['Status']['State']
        
        if status in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
            break
            
        print(f"Query status: {status}")
        time.sleep(2)
    
    if status != 'SUCCEEDED':
        raise Exception(f"Query failed: {response['QueryExecution']['Status']['StateChangeReason']}")
    
    # Get query results
    results_response = athena_client.get_query_results(QueryExecutionId=query_id)
    
    # Convert results to DataFrame
    columns = [col['VarCharValue'] for col in results_response['ResultSet']['Rows'][0]['Data']]
    rows = []
    
    for row in results_response['ResultSet']['Rows'][1:]:
        row_data = [col.get('VarCharValue', '') for col in row['Data']]
        rows.append(row_data)
    
    df = pd.DataFrame(rows, columns=columns)
    return df

In [6]:
# Problem 1: Top 10 subreddits by comment count
print("=== Problem 1: Top 10 subreddits by comment count ===")

query1 = """
SELECT 
    subreddit, 
    COUNT(*) as comment_count
FROM "AwsDataCatalog"."a05"."a05"
WHERE subreddit IS NOT NULL 
    AND subreddit != ''
GROUP BY subreddit
ORDER BY comment_count DESC
LIMIT 10
"""

df1 = execute_athena_query(query1)
df1.to_csv('prob1_results.csv', index=False)
print("Problem 1 completed. Results saved to prob1_results.csv")
print(df1.head())


=== Problem 1: Top 10 subreddits by comment count ===
Started query execution: de1683af-a608-46ac-abf9-a91429019283
Query status: QUEUED
Problem 1 completed. Results saved to prob1_results.csv
       subreddit comment_count
0      AskReddit        177871
1       facepalm         83841
2  AmItheAsshole         81354
3            nba         60028
4        TrueFMK         59138


In [7]:
# Problem 2: 10 random rows from comments table
print("\n=== Problem 2: 10 random rows ===")

query2 = """
SELECT *
FROM "AwsDataCatalog"."a05"."a05"
ORDER BY RAND()
LIMIT 10
"""

df2 = execute_athena_query(query2)
df2.to_csv('prob2_results.csv', index=False)
print("Problem 2 completed. Results saved to prob2_results.csv")
print(df2.head())


=== Problem 2: 10 random rows ===
Started query execution: e80af15d-8b2c-4960-8ce1-9054dd3e9189
Query status: QUEUED
Query status: RUNNING
Query status: RUNNING
Query status: RUNNING
Query status: RUNNING
Problem 2 completed. Results saved to prob2_results.csv
                 author author_flair_css_class      author_flair_text  \
0              shinneui                         :Ravenclaw: Ravenclaw   
1       chiraqmusicwiki                                                 
2  Affectionate-Air7554                                                 
3       slightlybearish                                                 
4              Honestnt                                                 

                                                body controversiality  \
0  Was he actually drunk? I seem to remember that...                0   
1  they don’t false claim like that fr, most garl...                0   
2  First of all, let’s stop and take a breath. Fe...                0   
3  Anyo

In [8]:
# Problem 3: Comments per day per hour
print("\n=== Problem 3: Comments per day per hour ===")

query3 = """
SELECT 
    DATE_FORMAT(FROM_UNIXTIME(created_utc), '%Y-%m-%d') as comment_date,
    HOUR(FROM_UNIXTIME(created_utc)) as comment_hour,
    COUNT(*) as comment_count
FROM "AwsDataCatalog"."a05"."a05"
WHERE created_utc IS NOT NULL
GROUP BY 
    DATE_FORMAT(FROM_UNIXTIME(created_utc), '%Y-%m-%d'),
    HOUR(FROM_UNIXTIME(created_utc))
ORDER BY comment_count DESC
"""

df3 = execute_athena_query(query3)
df3.to_csv('prob3_results.csv', index=False)
print("Problem 3 completed. Results saved to prob3_results.csv")
print(df3.head())


=== Problem 3: Comments per day per hour ===
Started query execution: 6c21911b-3e4d-4e6d-a11b-446c528b01d8
Query status: QUEUED
Problem 3 completed. Results saved to prob3_results.csv
  comment_date comment_hour comment_count
0   2023-06-01           16        442534
1   2023-06-01           17        439124
2   2023-06-01           18        438994
3   2023-06-01           19        433847
4   2023-06-01           20        431052


In [9]:
# Problem 4: Top 10 subreddits by average score
print("\n=== Problem 4: Top 10 subreddits by average score ===")

query4 = """
SELECT 
    subreddit,
    AVG(score) as avg_score
FROM "AwsDataCatalog"."a05"."a05"
WHERE subreddit IS NOT NULL 
    AND subreddit != ''
    AND score IS NOT NULL
GROUP BY subreddit
HAVING COUNT(*) >= 100  -- Only include subreddits with at least 100 comments
ORDER BY avg_score DESC
LIMIT 10
"""

df4 = execute_athena_query(query4)
df4.to_csv('prob4_results.csv', index=False)
print("Problem 4 completed. Results saved to prob4_results.csv")
print(df4.head())


=== Problem 4: Top 10 subreddits by average score ===
Started query execution: e4edc035-bee6-43fe-a06f-3de28f762985
Query status: QUEUED
Problem 4 completed. Results saved to prob4_results.csv
               subreddit           avg_score
0                Fauxmoi   58.81477298396205
1  BestofRedditorUpdates  55.067522838607175
2                 movies   52.95817774458551
3             rpdrcringe  52.891304347826086
4     nevertellmetheodds  52.321100917431195


In [10]:
# Problem 5: Most controversial comments in r/datascience
print("\n=== Problem 5: Most controversial comments in r/datascience ===")

query5 = """
SELECT 
    author,
    body,
    score,
    controversiality
FROM "AwsDataCatalog"."a05"."a05"
WHERE subreddit = 'datascience'
    AND controversiality = 1
    AND body IS NOT NULL
    AND body NOT IN ('[deleted]', '[removed]')
ORDER BY score ASC
LIMIT 5
"""

df5 = execute_athena_query(query5)
df5.to_csv('prob5_results.csv', index=False)
print("Problem 5 completed. Results saved to prob5_results.csv")
print(df5.head())


=== Problem 5: Most controversial comments in r/datascience ===
Started query execution: 64e273cd-f3a3-4d41-bb38-80902b0288e2
Query status: QUEUED
Query status: RUNNING
Problem 5 completed. Results saved to prob5_results.csv
        author                                               body score  \
0       MrEloi  Why did you continue after, say, the first 30 ...    -4   
1   miciomacho  You guys, you know, Nate Silver told me that j...    -2   
2       Tam27_  Do you have any templates that you use to mess...    -1   
3    bassabyss  I’m surprised no one has mentioned Lex Fridman...    -1   
4  Odd-One8023  It's against the rules of the subreddit. On to...     1   

  controversiality  
0                1  
1                1  
2                1  
3                1  
4                1  


In [12]:
# SQL query for extracting high-quality AI/GenAI comments
query6a = """
SELECT
    subreddit,
    author,
    body,
    score,
    created_utc,
    controversiality
FROM "AwsDataCatalog"."a05"."a05"
WHERE (
    REGEXP_LIKE(LOWER(subreddit), '^(artificial|artificialintelligence|ai|machinelearning|ml|deeplearning)$') OR
    REGEXP_LIKE(LOWER(subreddit), '^(chatgpt|openai|gpt|claude|anthropic|bard)$') OR
    REGEXP_LIKE(LOWER(subreddit), '^(midjourney|stablediffusion|dalle|dalle2|aiart|comfyui)$') OR
    REGEXP_LIKE(LOWER(subreddit), '^(tensorflow|pytorch|keras|huggingface|localllama)$') OR
    REGEXP_LIKE(LOWER(subreddit), '^(singularity|agi|automation|aiethics)$') OR
    REGEXP_LIKE(LOWER(subreddit), '.*artificial.*intelligence.*') OR
    REGEXP_LIKE(LOWER(subreddit), '.*machine.*learning.*') OR
    REGEXP_LIKE(LOWER(subreddit), '.*deep.*learning.*') OR
    REGEXP_LIKE(LOWER(subreddit), '.*generative.*ai.*') OR
    REGEXP_LIKE(LOWER(subreddit), '.*chat.*gpt.*') OR
    REGEXP_LIKE(LOWER(subreddit), '.*stable.*diffusion.*')
)
AND LENGTH(body) BETWEEN 100 AND 1000
AND score >= 2
AND body NOT IN ('[deleted]', '[removed]')
AND body IS NOT NULL
AND author IS NOT NULL
AND author != 'AutoModerator'
ORDER BY score DESC, created_utc DESC
"""

# Save the SQL query to file for Part A
with open('problem6_query.sql', 'w') as f:
    f.write(query6a)